In [176]:
import pandas as pd
import numpy as np
df= pd.read_csv('/Users/ankit/Desktop/Springboard dox/Python/2018_less.csv',low_memory=False)


In [177]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Columns: 150 entries, id to settlement_term
dtypes: float64(63), int64(57), object(30)
memory usage: 11.4+ MB
None


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,145217616,NaN,10000,10000,10000,36 months,10.33%,324.23,B,B1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,145248657,NaN,4000,4000,4000,36 months,23.4%,155.68,E,E1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,145638579,NaN,5000,5000,5000,36 months,17.97%,180.69,D,D1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,145592722,NaN,23000,23000,23000,60 months,20.89%,620.81,D,D4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,144858623,NaN,8000,8000,8000,36 months,23.4%,311.35,E,E1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
print(df.shape)

(9998, 150)


In [179]:
#Drop columns with data NaN

In [180]:
df=df.dropna(axis=1)

In [181]:
df.shape

(9998, 89)

In [182]:
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,home_ownership,...,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,145217616,10000,10000,10000,36 months,10.33%,324.23,B,B1,MORTGAGE,...,4,91.3,0,0,367828,61364,20900,54912,N,N
1,145248657,4000,4000,4000,36 months,23.4%,155.68,E,E1,RENT,...,9,95.0,0,0,98655,66926,21900,71555,N,N
2,145638579,5000,5000,5000,36 months,17.97%,180.69,D,D1,MORTGAGE,...,0,100.0,0,0,136927,11749,13800,10000,N,N
3,145592722,23000,23000,23000,60 months,20.89%,620.81,D,D4,RENT,...,3,100.0,0,0,7500,976,3300,0,N,N
4,144858623,8000,8000,8000,36 months,23.4%,311.35,E,E1,OWN,...,2,100.0,1,0,199744,31078,2300,32206,N,N


In [184]:
#rename the column names

df = df.rename(columns={"loan_amnt": "loan_amount", "funded_amnt": "funded_amount", "funded_amnt_inv": "investor_funds",
                       "int_rate": "interest_rate", "annual_inc": "annual_income"})


In [185]:
df.head()
print(type(df['interest_rate'][0]))

<class 'str'>


In [190]:
#removing percentage sign from the interest_rate
df.interest_rate = df.interest_rate.str.replace('%', '')
print(df.interest_rate.head())

0    10.33
1     23.4
2    17.97
3    20.89
4     23.4
Name: interest_rate, dtype: object


In [191]:
#remove months from term colums
df.term = df.term.str.replace('months', '')
df.term.head()

0    36 
1    36 
2    36 
3    60 
4    36 
Name: term, dtype: object

In [108]:
#selecting unique column as index

df['id'].is_unique

True

In [112]:
df.set_index('id')

,loan_amount,funded_amount,investor_funds,term,interest_rate,installment,grade,sub_grade,home_ownership,annual_income,...,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,
145217616,10000,10000,10000,36 months,10.33%,324.23,B,B1,MORTGAGE,280000.0,...,4,91.3,0,0,367828,61364,20900,54912,N,N
145248657,4000,4000,4000,36 months,23.4%,155.68,E,E1,RENT,90000.0,...,9,95.0,0,0,98655,66926,21900,71555,N,N
145638579,5000,5000,5000,36 months,17.97%,180.69,D,D1,MORTGAGE,59280.0,...,0,100.0,0,0,136927,11749,13800,10000,N,N
145592722,23000,23000,23000,60 months,20.89%,620.81,D,D4,RENT,68107.0,...,3,100.0,0,0,7500,976,3300,0,N,N
144858623,8000,8000,8000,36 months,23.4%,311.35,E,E1,OWN,43000.0,...,2,100.0,1,0,199744,31078,2300,32206,N,N
145606176,32075,32075,32075,60 months,11.8%,710.26,B,B4,MORTGAGE,150000.0,...,2,100.0,0,0,360433,56635,20000,80125,N,N
145647242,9600,9600,9600,36 months,12.98%,323.37,B,B5,MORTGAGE,35704.0,...,0,95.5,0,0,6500,748,4200,0,N,N
145631930,30000,30000,30000,60 months,18.94%,777.23,D,D2,MORTGAGE,90000.0,...,6,95.0,1,0,372872,99468,15000,94072,N,N
145640422,2500,2500,2500,36 months,13.56%,84.92,C,C1,RENT,55000.0,...,3,100.0,1,0,60124,16901,36500,18124,N,N
